In [ ]:
#Have in mind that data are not normaly distributed, futher will be apply non-parametical alternatives.

In [21]:
import pandas as pd
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
from scipy.stats import mannwhitneyu
import numpy as np
from scipy.stats import kruskal

In [2]:
data = pd.read_csv('cleaned_dataset.csv')

In [3]:
data.head(3)

,bet_id,user_id,bet_type,sport,is_win,odds_log,stake_log,gain_log,GGR_log
0,bet_00001,3848,multiple,Ice Hockey,False,2.345645,2.684440,0.000000,6.894974
1,bet_00002,153,single,Football,False,2.281361,5.519258,0.000000,6.623401
2,bet_00003,1527,single,Football,True,0.916291,1.504077,1.832581,6.910452


In [9]:
data.shape

(100000, 9)

In [ ]:
# Hi-squared test of independence
# Research question: Wheater the proporsion of users who bet on football and won is equal to the proporsion who lost?

In [7]:
# Filter football bets
football_bets = data[data['sport'] == 'Football']

In [8]:
# Contingency table for wins and losses
contingency_table = pd.crosstab(index=football_bets['is_win'], columns='count')

print(contingency_table)

col_0   count
is_win       
False   31216
True    18006


In [11]:
# counts of wins and losses
observed = contingency_table['count'].values

# Expected counts assuming equal proportion (half of total)
total = observed.sum()
expected = [total/2, total/2]

chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"Chi2 Statistic: {chi2_stat}")
print(f"P-value: {p_value}")

Chi2 Statistic: 3545.246028198773
P-value: 0.0


In [ ]:
# Hi-square test has very large value for test statistic and p=0 < 0.05. Null hypotesis wil be rejected, which means that proportion of
# wins and loses among users who are bet on football are not equal. The data show that proporsion of users who are lost is significantly different from 50-50 splt.  

In [ ]:
# Mann-Whitney test
# Example question: Does bet_type(sigle or multiple) differ by odds level?

In [16]:
bet_types = data['bet_type'].unique()
print(bet_types)

['multiple' 'single']


In [ ]:
#want to compare bet_type 'A' vs 'B'
group1 = data[data['bet_type'] == bet_types[0]]['odds_log']
group2 = data[data['bet_type'] == bet_types[1]]['odds_log']

In [18]:
stat, p = mannwhitneyu(group1, group2, alternative='two-sided')

print(f"Mann-Whitney U statistic: {stat}")
print(f"P-value: {p}")

Mann-Whitney U statistic: 1414351799.0
P-value: 0.0


In [ ]:
# Null hypothesis: distribution of odds for the both bets type is same.
# p =0, <0.05. Distrinution of odds level is significantly differ between bet_types.

In [20]:
# Effect of size:
U, p = mannwhitneyu(group1, group2, alternative='two-sided')

n1 = len(group1)
n2 = len(group2)

# Calculate mean and std of U under null hypothesis
mu_U = n1 * n2 / 2
sigma_U = np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12)

# Calculate Z-score
Z = (U - mu_U) / sigma_U

# Calculate effect size r
N = n1 + n2
r = abs(Z) / np.sqrt(N)


print(f"Effect size r: {r:.3f}")

Effect size r: 0.201


In [ ]:
# A small effect of size=0.201.

In [ ]:
# Kruskal-Walls test is non-parametric alternatives for one-way anova. Use to compare result of continue variable in three or more group.
# The result will be transform in rank and mean rang of each group will be compared.

In [ ]:
# will be used one continuour variable: GGR, one catorical variable sport.

In [23]:
# unique sports
sports = data['sport'].unique()

# list of GGR arrays, one per sport
groups = [data[data['sport'] == sport]['GGR_log'] for sport in sports]

In [24]:
stat, p = kruskal(*groups)

print(f"Kruskal-Wallis H statistic: {stat}")
print(f"P-value: {p}")

Kruskal-Wallis H statistic: 16.44372551121684
P-value: 0.17174716114041577


In [ ]:
# Null hypothesis: All groups have same distribution of GGR.
# p =0.17 >0.05. GGR does not vary significantly between sport.